# Исследование надёжности заёмщиков

Цель поекта - на основе статистике кредитного отдела определить влияние семейного положения и количества детей на возврат кредита в срок.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Загрузка файла с данными

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
# Изучаем общую информацию по датасету.
df.info()
df.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
# Просмотр уникальных совпадений значений столбцов, исключая столбцы 'days_employed', total_income'
for i in list(df):
    if i not in ('days_employed', 'total_income'):
        print('Данные по столбцу -', i)
        print(df[i].value_counts(), '\n')

# Вывод информации о значениях стобца 'days_employed'
print(
      "Информация о значениях в столбце 'days_employed':", '\n',
      'Количество положительных значений', df[df['days_employed'] > 0]['days_employed'].count(), '\n',
      'Количество отрицательных значений', df[df['days_employed'] < 0]['days_employed'].count(), '\n',
      'Максимальное положительное значение', df[df['days_employed'] > 0]['days_employed'].max(), '\n',
      'Минимальное отрицательное значение', df[df['days_employed'] < 0]['days_employed'].min(), '\n',
      'Количество значений равных 0', df[df['days_employed'] == 0]['days_employed'].count(), '\n',
      )

# Вывод информации о значениях стобца 'total_income'   
print(
      "Информация о значениях в столбце 'total_income':", '\n',
      'Средее значение', df['total_income'].mean().round(), '\n',
      'Медиана', df['total_income'].median().round(), '\n',
      'Максимальное значение:', df['total_income'].max().round(), '\n',
      'Количество значение равных:', df[df['total_income'] == 0]['total_income'].count()
     )

Данные по столбцу - children
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64 

Данные по столбцу - dob_years
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64 

Данные по столбцу - education
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ             

In [3]:
# Вычисляем количество дубликатов
print('Количество дубликатов:', df.duplicated().sum(), '\n')

# Проверка наличия пропусков
print('Количество пропусков:', df.isna().sum(), '\n')
print('Количество строк с пропусками:', df.isna().any(axis=1).sum())

Количество дубликатов: 54 

Количество пропусков: children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64 

Количество строк с пропусками: 2174


**Вывод**

Датасет содержит 21525 строк и 12 столбцов.  
В датасете 54 строки - дубликаты. Дубликаты могли возникнуть при формировании датасета из нескольких баз, где были совпадающие записи.  
В 2174 строках пропуски по столбцам 'days_employed' и 'total_income'. Пропуски в данных могли возникнуть при выгрузке из базы данных в датасет.  
Столбец 'days_employed' содержит как положительные так и отрицательные значания. Положительные значения аномально большие и требуют корректировки.  
В столбце 'children' присутствуют некорректные данные, очевидно связанные с ошибками ввода.  
В столбце 'dob_years' присутствуют ошибочные значения равные нулю.  
В столбце 'education' присутствуют поля заполненные строчными и прописными буквами.  

Датасет содержит 21525 строк и 12 столбцов.  
В датасете 54 строки - дубликаты. Дубликаты могли возникнуть при формировании датасета из нескольких баз, где были совпадающие записи.  
В 2174 строках пропуски по столбцам 'days_employed' и 'total_income'. Пропуски в данных могли возникнуть при выгрузке из базы данных в датасет.  
В столбце 'children' присутствуют аномальные значения, очевидно связанные с ошибками ввода. Количество аномальных значений не превышает 0,6% от числа значений в столбце.  
Столбец 'days_employed' содержит как положительные так и отрицательные значания.Положительные начения составляют 17,8% от числа значений в столбце. Положительные значения аномально большие и требуют корректировки.  
В столбце 'dob_years' присутствуют ошибочные значения равные нулю. Количество значений равных нулю не превышает 0,5% от числа значений в столбце. 
В столбце 'gender' присутствует одно аномальное значение - 'XNA'.
В столбце 'total_income' отсутствуют признаки аномальных значений.  
В столбце 'education' присутствуют поля заполненные строчными и прописными буквами. 
В остальных столбцах аномальные значения отсутствуют.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
# Устаняем ошибку в больших положительных значениях столбца 'days_employed'
# Делаем предположение, что масимальный общий трудовай стаж клиентов с отрицацательными показателями 'days_employed'
# приблизительно равен масимальному трудовому стажу клиентов с положительными 'days_employed'

# Вычисляем коэффициент на который нужно уменьшить отрицательные значения 'days_employed'
k = df['days_employed'].max() / - df['days_employed'].min()

# вычисляем новые отрицательные значения столбца 'days_employed'
df['days_employed'] = df.days_employed.apply(lambda x: x/k if x > 0 else x)

# Приводим все числа столбца 'days_employed' к положительным значениям
df['days_employed'] = df['days_employed'].abs()

# Функция для категоризации клиентов по возрасту
def grp_years(years):
    if 19 <= years < 30:
        return '19-29'
    if 30 <= years < 40:
        return '30-39'
    if 40 <= years < 50:
        return '40-49'
    if 50 <= years < 60:
        return '50-59'
    if 60 <= years < 70:
        return '60-69'
    if 70 <= years < 80:
        return '70-79'
    return 'unknown'

# Добавляем столбец с возрастными категориями
df['age_group'] = df['dob_years'].apply(grp_years)

# Заполняем пропущенные значения столбца 'days_employed' медианами значений 'days_employed', сгрупированых  по столбцу 'age_group'
df['days_employed'] = df.groupby('age_group').days_employed.apply(lambda r: r.fillna(r.median()))

# Заполняем пропущенные значения столбца 'total_income' медианами значений 'total_income' сгрупированых по столбцу 'income_type'
df['total_income'] = df.groupby('income_type').total_income.apply(lambda r: r.fillna(r.median()))

# Проверка на наличие пропусков
print('Количество пропущенных значений после обработки:', df.isna().sum(), sep='\n')

Количество пропущенных значений после обработки:
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64


**Вывод**

Для заполнения пропущенных значений в столбце 'days_employed' выполнена категоризаци заемщиков по возрасту, затем пропущенные значения заполнены медианой 'days_employed' категории в которую входит возраст заемщика.  
Пропущенные значений в столбце 'total_income' были заполнены медианами 'total_income' типа занятости заемщика.  
После заполнения пропущенных данных проверка показала, что пропущенных значений нет.

### Замена типа данных

In [5]:
# Замена типа данных в столбцах 'days_employed', 'total_income'
df['days_employed'] = df['days_employed'].round().astype(int).abs()
df['total_income'] = df['total_income'].round().astype(int)

# Замена типа данных в столбце 'education'
df['education'] = df['education'].str.lower()
print("Уникальные значения столбца 'education' после преобразования", '\n', df['education'].unique(), '\n')

# Удаление строк содержащие в столбце 'children' значения -1, 20
df = df[(df['children'] != -1) & (df['children'] != 20)]
print("Уникальные значения столбца 'children' после очистки", '\n', df['children'].unique(), '\n')

# Удаление строки содержащую в столбце 'gender' значения 'XNA'
df = df[df['gender'] != 'XNA']
print("Уникальные значения столбца 'gender' после очистки", '\n', df['gender'].unique(), '\n')

df.info()

Уникальные значения столбца 'education' после преобразования 
 ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень'] 

Уникальные значения столбца 'children' после очистки 
 [1 0 3 2 4 5] 

Уникальные значения столбца 'gender' после очистки 
 ['F' 'M'] 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21401 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21401 non-null  int64 
 1   days_employed     21401 non-null  int64 
 2   dob_years         21401 non-null  int64 
 3   education         21401 non-null  object
 4   education_id      21401 non-null  int64 
 5   family_status     21401 non-null  object
 6   family_status_id  21401 non-null  int64 
 7   gender            21401 non-null  object
 8   income_type       21401 non-null  object
 9   debt              21401 non-null  int64 
 10  total_income      21401 non-null  int64 
 11  purpose        

**Вывод**

В столбце 'days_employed' произведено округление до целых чисел и заменен тип данных: числа с плавающей точкой преобразованы до целых положительных чисел.   
В столбце 'total_income' произведено округление до целых чисел и заменен тип данных: числа с плавающей точкой преобразованы до целых чисел.  
Замена типа данных выполнена методом '.astype()' так как этот метод позволяет нам четко указать тип, в который мы хотим преобразовать.  
В столбце 'education' данные преобразованы  в строки содержаще только прописные буквы
Так как количество аномальных значений в столбце 'children' не превышает 0,6%, в датафрейме удалены строки с аномальными значениями в столбце 'children'  
Так как в столбце 'gender' всего одно аномальное значение, в датафрейме удалена строка с аномальными значениями в столбце 'gender'  

### Обработка дубликатов

In [6]:
# Удаляем дубликаты строк и после проверяем их отстутствие
df = df.drop_duplicates().reset_index(drop=True)
print('Количество дубликатов:', df.duplicated().sum())

Количество дубликатов: 0


### Лемматизация

In [7]:
# Вывод списка значений столбца 'purpose' для определения групп целей кредита
print(df['purpose'].value_counts())

#Создание функции, которая группирует цели кредита
from pymystem3 import Mystem
m = Mystem()
def get_purpose_grp(p):
    lemma = m.lemmatize(p)
    if 'автомобиль' in lemma:
        return 'автомобиль'
    elif 'недвижимость' in lemma:
        return 'недвижимость'    
    elif ('жилье' in lemma) and ('ремонт' in lemma):
        return 'ремонт жилья'
    elif ('жилье' in lemma):
        return 'жилье'
    elif 'образование' in lemma:
        return 'образование'
    elif 'свадьба' in lemma:
        return 'свадьба'
    return 'цель не определена'

свадьба                                   790
на проведение свадьбы                     763
сыграть свадьбу                           760
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с жильем                         647
операции с коммерческой недвижимостью     645
жилье                                     641
покупка жилья                             640
покупка жилья для семьи                   637
недвижимость                              631
строительство собственной недвижимости    628
операции со своей недвижимостью           623
строительство жилой недвижимости          620
строительство недвижимости                619
покупка своего жилья                      619
покупка недвижимости                      615
ремонт жилью                              604
покупка жилой недвижимости                602
на покупку своего автомобиля              504
заняться высшим образованием      

**Вывод**

Создана функция для лемматизации и категоризации данных в столбце 'purpose'

### Категоризация данных

In [8]:
# Категоризация данных столбца 'purpose'
df['purpose_grp'] = df['purpose'].apply(get_purpose_grp)
df['purpose_grp'].unique()

array(['жилье', 'автомобиль', 'образование', 'свадьба', 'недвижимость',
       'ремонт жилья'], dtype=object)

In [9]:
# Категоризация данных столбца 'children'
df['children_true'] = df['children'] > 0
df['children_true'].unique()

array([ True, False])

In [10]:
print("Исходный список уникальных значений столбца 'family_status':", '\n', df['family_status'].unique())
# Категоризация данных столбца 'family_status'
df['married'] = (df['family_status'] == 'женат / замужем') | (df['family_status'] == 'гражданский брак')
df['married'].unique()

Исходный список уникальных значений столбца 'family_status': 
 ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


array([ True, False])

In [11]:
# Категоризация данных столбца 'total_income'
income_max = df['total_income'].max()
def income_grp(x):
    if x < income_max / 5:
        return 'низкий'
    elif x < income_max / 5 * 2:
        return 'средний'
    elif x < income_max / 5 * 3:
        return 'повышенный'
    elif x < income_max / 5 * 4:
        return 'высокий'
    return 'очень высокий'
df['income_level'] = df['total_income'].apply(income_grp)
df['income_level'].unique()

array(['низкий', 'средний', 'повышенный', 'высокий', 'очень высокий'],
      dtype=object)

**Вывод**

Проведена категоризация данных в столбцах: 'purpose', 'children', 'family_status', 'total_income'.  
В датасет добавлены  новые столбцы с категориями: 'purpose_grp', 'children_true', 'married', 'income_level'

## Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [12]:
print('Распределение клиентов по наличию детей:', df['children_true'].value_counts(), sep='\n')
df.groupby('children_true')['debt'].mean()

Распределение клиентов по наличию детей:
False    14090
True      7240
Name: children_true, dtype: int64


children_true
False    0.075444
True     0.092403
Name: debt, dtype: float64

**Вывод**

Существует зависимость между наличием детей и возвратом кредита в срок.
Вероятность просрочки кредита клиентов без детей составляет 7,5%, а для клиентов с детьми 9,2%. Вероятность просрочки кредита клиентов с детьми на 23% выше, чем у клиентов без детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [13]:
print('Распределение клиентов по семейному положению:', df['married'].value_counts(), sep='\n')
df.groupby('married')['debt'].mean()

Распределение клиентов по семейному положению:
True     16394
False     4936
Name: married, dtype: int64


married
False    0.085089
True     0.080029
Name: debt, dtype: float64

**Вывод**

Существует зависимость между между семейным положением и возвратом кредита в срок.
Вероятность просрочки кредита клиентов не состоящих в браке составляет 8,5%, а для клиентов состоящих в браке 7,9%. Вероятность просрочки кредита клиентов не состоящих в браке выше на 8% чем у клиентов состоящих в браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [14]:
print('Распределение клиентов по уровню дохода:', df['income_level'].value_counts(), sep='\n')
df.groupby('income_level')['debt'].mean()

Распределение клиентов по уровню дохода:
низкий           21006
средний            294
повышенный          22
высокий              6
очень высокий        2
Name: income_level, dtype: int64


income_level
высокий          0.000000
низкий           0.081596
очень высокий    0.500000
повышенный       0.045455
средний          0.054422
Name: debt, dtype: float64

**Вывод**

Существует зависимость между доходом клиента и риском не возврата кредита в срок.
Чем ниже доход клиента, тем выше риск не возврата кредита в срок.  
Высокий уровень риска 50% у клиентов с уровнем дохода 'очень высокий' может быть случаен, так как таких клиентов всего двое.

- Как разные цели кредита влияют на его возврат в срок?

In [15]:
print('Распределение клиентов по целям кредита:', df['purpose_grp'].value_counts(), sep='\n')
df.groupby('purpose_grp')['debt'].mean()

Распределение клиентов по целям кредита:
недвижимость    6313
автомобиль      4279
образование     3988
жилье           3833
свадьба         2313
ремонт жилья     604
Name: purpose_grp, dtype: int64


purpose_grp
автомобиль      0.093480
жилье           0.071224
недвижимость    0.074766
образование     0.092528
ремонт жилья    0.057947
свадьба         0.079118
Name: debt, dtype: float64

**Вывод**

Наибольшая вероятность просрочки кредита имеют клиенты берущие кредит для покупки автомобиля, оплаты образования и для оплаты затрат на свадьбу.

## Общий вывод

    В проекте выполнено исследование надежности заемщиков по нескольким критериям. 
    Из предоставленного файла с датасетом был создан датафрейм.  Был проведен анализ датафрейма на корретность значений, целостность данных. При анализе информации содержащейся в датафрейме были обнаружены аномальные данные, пропуски, дубликаты. Была проведена работа по исправлению некорректных данных. Аномальные данные, содержание которых не превышало 5% были удалены. Числовые данные были преобразованы в целые положительные числа. Строковые данные преобразованы в текст, содержащий только строчные символы. Из датафрейма удалены строки с дубликатами. 
    Для проведения дальнейшего анализа датафрейма, была проведена лемматизация значений столбца содержащего цели кредита клиентов, затем проведена категоризация целей кредита заемщиков.  
    Были даны ответы заказчику на влияние наличия детей, дохода, семейного положения, целей кредита на риск просрочки кредита.  
    Наибольшее влияние на надежность заемщика оказывает наличие детей. Вероятность просрочки кредита клиентов с детьми на 23% выше, чем у клиентов без детей.  Также влияет на риск просрочки кредита доход, семейное положение заемщика и цели кредита.  
    Для более точного и надежного определения риска просрочки кредита заказчику рекомендовано получить более целые и корректные данные о клиентах. Также для оценки риска рекомедовано проанализировать влияние комбинации нескольких параметров заемщиков на вероятность просрочки кредита.